# HETDEX AGN Catalog 1

In [1]:
import os.path as op
import numpy as np
from astropy.io import ascii, fits
from astropy.io.fits import getdata
from astropy.table import Table, Column,join

In [2]:
# fill in the path to the HETDEX AGN Catalog 1 file

path_to_agn1 = "/home/jovyan/Hobby-Eberly-Public/HETDEX/catalogs/agn_catalog_v1.0/"

if op.exists(path_to_agn1):
    pass
else:
    path_to_agn1 = "/corral-repl/utexas/Hobby-Eberly-Telesco/public/HETDEX/catalogs/agn_catalog_v1.0/"

# Check basic info of the fits file
### There are six extensions, 1 & 4 are table extensions, 2,3 & 5,6 are image extensions for spectra and their errors.

There are three major differences between ext2&3 and ext5&6.

(1) We provide two types of spectra with their errors in ext 2&3 and ext5&6: 
As an IFU survey, some spacially extended AGNs can be spacially resolved with multiple detections in the hetdex catalog. We do flux weighted friend-of-friend grouping for all AGN detectids with linking length of delta_r=5 arcsec, and delta_redshift = 0.1 to make the unqiue AGN catalog (extension 1). For each AGN, we pick up the detectid closest to the fof center as detectid_best.

ext 2&3 are the spectra for the detectid_best in the hetdex catalog. \
ext5&6 are extracted spectra at the flux weighted friend-of-friend center.

(2) Aonther difference between ext2&3 and ext5&6 is the number of rows are different. \
ext2&3 have 5322 rows, one row for one agn, arranged in the same order with extension 1.\
ext5&6 have 6004 rows. This is because there are some AGNs that have repeat observations (identified by "shotid" in the table of extension 4). We provide all the spectra for repeated observations in ext 5&6. In the table of extension 4, each observation has one row, and the row number of ext5&6 are arranged following extension4.

(3) A minor difference is:\
    ext2&3 spectra with errors are applied with universal extinction correction of E(B-V)=0.02.\
    ext5&6 are raw data with no extinction applied.\
    The extinction correction only makes a <10% difference.

In [3]:
fname = op.join( path_to_agn1, 'hetdex_agn.fits')
hdul = fits.open(fname)
hdul.info()

Filename: /corral-repl/utexas/Hobby-Eberly-Telesco/public/HETDEX/catalogs/agn_catalog_v1.0/hetdex_agn.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  basic_info    1 BinTableHDU    735   5322R x 159C   [K, E, E, E, E, K, 10A, K, K, E, E, E, K, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, 38A, E, E, E, E, 16A, E, E, E, E, 16A, E, E, E, E, 16A, E, 15A, E, E, K, K, K, E, K, E]   
  2  flux_array    1 ImageHDU        11   (1036, 5322)   float32   
  3  error_array    1 ImageHDU        11   (1036, 5322)   float32   
  4  repeat_info    1 BinTableHDU     15   6004R x 3C   [K, K, K]   
  5  flux_array_repeat  

# Read extension 1 as a table
extension 1 has the basic information for each unique AGN, one row per AGN.

In [4]:
agn = Table.read( fname, format = 'fits', hdu = 1 )
agn

agnid,ra,dec,z,z_er,zflag,field,nshots,detectid_best,ra_best,dec_best,roff,nmem,alpha,alpha_er,fpl0,fpl0_er,slope,slope_er,intercept,intercept_er,L1350,L1350_er,L1450,L1450_er,L3000,L3000_er,L5100,L5100_er,fwhm_LyA,fwhm_LyA_er,ew_LyA,ew_LyA_er,flux_LyA,flux_LyA_er,cont_LyA,cont_LyA_er,snr_LyA,snr_LyA_er,chi2_LyA,chi2_LyA_er,fwhm_NV,fwhm_NV_er,ew_NV,ew_NV_er,flux_NV,flux_NV_er,cont_NV,cont_NV_er,fwhm_SiO,fwhm_SiO_er,ew_SiO,ew_SiO_er,flux_SiO,flux_SiO_er,cont_SiO,cont_SiO_er,snr_SiO,snr_SiO_er,chi2_SiO,chi2_SiO_er,fwhm_CIV,fwhm_CIV_er,ew_CIV,ew_CIV_er,flux_CIV,flux_CIV_er,cont_CIV,cont_CIV_er,snr_CIV,snr_CIV_er,chi2_CIV,chi2_CIV_er,fwhm_HeII,fwhm_HeII_er,ew_HeII,ew_HeII_er,flux_HeII,flux_HeII_er,cont_HeII,cont_HeII_er,snr_HeII,snr_HeII_er,chi2_HeII,chi2_HeII_er,fwhm_CIII,fwhm_CIII_er,ew_CIII,ew_CIII_er,flux_CIII,flux_CIII_er,cont_CIII,cont_CIII_er,snr_CIII,snr_CIII_er,chi2_CIII,chi2_CIII_er,fwhm_MgII,fwhm_MgII_er,ew_MgII,ew_MgII_er,flux_MgII,flux_MgII_er,cont_MgII,cont_MgII_er,snr_MgII,snr_MgII_er,chi2_MgII,chi2_MgII_er,fwhm_OII,fwhm_OII_er,ew_OII,ew_OII_er,flux_OII,flux_OII_er,cont_OII,cont_OII_er,snr_OII,snr_OII_er,chi2_OII,chi2_OII_er,fwhm_OVI,fwhm_OVI_er,ew_OVI,ew_OVI_er,flux_OVI,flux_OVI_er,cont_OVI,cont_OVI_er,snr_OVI,snr_OVI_er,chi2_OVI,chi2_OVI_er,fiberid,r_aper,r_aper_er,r_radius,r_depth,r_cat,g_aper,g_aper_er,g_radius,g_depth,g_cat,u_aper,u_aper_er,u_radius,u_depth,u_cat,rsep_dr14q,id_dr14q,z_dr14q,rsep_dr16,plate_dr16,fiber_dr16,mjd_dr16,z_dr16,sflag,apcor
,deg,deg,,,,,,,deg,deg,arcsec,,,,,,,,,,1e+44 erg / s,1e+44 erg / s,1e+44 erg / s,1e+44 erg / s,1e+44 erg / s,1e+44 erg / s,1e+44 erg / s,1e+44 erg / s,km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),,,,,km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),,,,,km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),,,,,km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),,,,,km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),,,,,km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),,,,,km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),,,,,km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),,,,,,mag,mag,arcsec,arcsec,,mag,mag,arcsec,arcsec,,mag,mag,arcsec,arcsec,,arcsec,,,arcsec,,,,,,
int64,float32,float32,float32,float32,int64,bytes10,int64,int64,float32,float32,float32,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes38,flo

### check the header of the table

In [5]:
agn.info

<Table length=5322>
     name      dtype              unit                               description                  
------------- ------- ---------------------------- -----------------------------------------------
        agnid   int64                                   artificial sequential numbers for each AGN
           ra float32                          deg    RA  of the AGN (center of flux weighted fof)
          dec float32                          deg    DEC of the AGN (center of flux weighted fof)
            z float32                                                                     redshift
         z_er float32                                                            error of redshift
        zflag   int64                                        zflag=0/1: 1 confirmed z; 0 guessed z
        field bytes10                                                   field in the HETDEX survey
       nshots   int64                                                number of repeat obs

## Read extension 2 in as a 2-d array

In [6]:
from astrowidgets import ImageWidget

#hdu2, hdr2 = getdata( fname, 2, header = True )
print(hdul[2].header)
im    = hdul[2].data # read spectra in from extension 2
im_er = hdul[3].data # read errors  in from extension 3 if needed

imw = ImageWidget()
imw.cuts = (-1, 25)
imw.load_array(im)
display(imw)

XTENSION= 'IMAGE   '           / Image extension                                BITPIX  =                  -32 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                 1036                                                  NAXIS2  =                 5322                                                  PCOUNT  =                    0 / number of parameters                           GCOUNT  =                    1 / number of groups                               EXTNAME = 'flux_array'                                                          HIERARCH wave_start = '3470.0 AA'                                               HIERARCH wave_step = '   2.0 AA'                                                HIERARCH flux_unit = '1.0e-17 * erg / (cm^2 * s * AA)'                          END                                                                                                                     

ImageWidget(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\x…

the information of the wave_array corresponds to the flux_array in each row can be found in the header: 

wave_start = 3470.0, wave_step = 2.0

In [7]:
hdul[2].header['wave_start'], hdul[2].header['wave_step'], hdul[2].header['flux_unit']

('3470.0 AA', '   2.0 AA', '1.0e-17 * erg / (cm^2 * s * AA)')

In [8]:
# prepare wavelength array for the spectra
wave_start = 3470.0
wave_step  = 2.0
n_elements = hdul[2].header['NAXIS1'] #1036
wave_arr = wave_start + wave_step * np.arange( 0, n_elements )
wave_arr

array([3470., 3472., 3474., ..., 5536., 5538., 5540.])

In [9]:
from IPython.display import Image
import matplotlib.pyplot as plt

# ............
w_CIIIa = 977.030
w_NIII  = 991.514
w_OVIa  = 1031.912
w_OVIb  = 1037.613
w_LyA   = 1215.24 
w_NV    = 1240.81
w_OI    = 1305.53
w_CIIa  = 1335.31
w_SiO   = 1399.8
w_CIV   = 1549.48
w_HeII  = 1640.40
w_OIII  = 1665.85
w_CIIIb = 1908.734
w_CIIb  = 2326.0
w_NeIV  = 2439.5
w_MgII  = 2799.117
w_OII   = 3729.875
w_NeVI  = 3426.85
w_NeIII = 3868.760
w_Hg    = 4341.68
# ............

fontsize = 15
xl, xh = 3450, 5550 

# plot some example spectra at the following redshifts
z_sel = np.array( [ 0.9, 1.3, 1.8, 2.2, 2.5, 3.9 ] )

for i in range(np.size(z_sel)):
    sel  = agn['z'] < z_sel[i]
    dtmp = im[sel]
    flux_arr = dtmp[0]
    
    fig, ax = plt.subplots( figsize = ( 10., 5.)  )
    
    plt.plot( wave_arr, flux_arr, 'k' )
    
    lines   = np.array( [  w_OVIa, w_OVIb, w_LyA,   w_NV,   w_SiO,  w_CIV, w_HeII,   w_CIIb, w_CIIIb,  w_MgII, w_OII, w_NeVI, w_NeIII  ] )
    labels  = np.array( [  ' ',     'OVI',  'LyA','    NV', 'Si+O',   'CIV', 'HeII',    'C2', 'C III]', 'MgII', 'OII', 'NeVI', 'NeIII' ] )    

    yl, yh = np.min(flux_arr), np.max(flux_arr)
    for i in range( np.size(lines) ):
        wline = lines[i] * ( 1 + agn[sel]['z'][0] )
        if( ( wline >= xl ) and (wline <= xh ) ):
            wline = np.full( 2, wline )     
            plt.text( wline[0]+10, 0.9 * yh, labels[i], color = 'blue', fontsize=14 )
            plt.plot( wline,  np.array([yl,yh]), color = 'blue', linestyle = '--', linewidth = 1.0 )
    
    plt.xlim( xl, xh )
    plt.tick_params( which = 'both', bottom=True, top=True, left=True, right=True )

    plt.xlabel('$\mathrm{\lambda_{obs}\ (\AA)}$',fontsize=fontsize)
    plt.ylabel('$\mathrm{f_{\lambda,obs}\ (10^{-17} ergs/s/cm^2/\AA)}$',fontsize=fontsize)
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)

    stitle = 'agnid=%d, detectid_best=%d, z=%.2f' % (agn[sel]['agnid'][0], agn[sel]['detectid_best'][0], agn[sel]['z'][0])
    plt.title( stitle, fontsize=fontsize )
    plt.show()
    #plt.savefig( 'tmp.png', format='png')
    #Image('tmp.png')

# read extension 4 in as a table of repeat observation info
### There are some repeat observations for some agns. In this table, each observation has one entry with a unique "shotid", nshots shows the number of repeat observations for each unique AGN.

In [10]:
tab = Table.read( fname, format = 'fits', hdu = 4 )
tab

agnid,nshots,shotid
int64,int64,int64
1,1,20191006024
2,1,20190504027
3,1,20190403022
4,2,20200414024
4,2,20200527024
5,2,20190501017
5,2,20190429016
6,1,20191024024
7,1,20171115010


In [11]:
sel = tab['nshots'] == 1
print("There are %d AGNs that are only observed once" % np.sum(sel))

sel = tab['nshots'] == 2
dtmp = np.unique( tab[sel]['agnid'] )
print("There are %d AGNs that are observed twice" % np.size(dtmp) )

sel = tab['nshots'] > 2
dtmp = np.unique( tab[sel]['agnid'] )
print("There are %d AGNs that are observed more than twice" % np.size(dtmp) )

There are 4799 AGNs that are only observed once
There are 437 AGNs that are observed twice
There are 74 AGNs that are observed more than twice


# Find all spectra and their error of a certain AGN in extension 5 and 6

### find some strong agns (big nmem) with secure redshifts (z!=0) with more than five repeat observations

In [12]:
sel1 = agn['nshots'] >= 5
sel2 = agn['zflag' ] != 0
sel = sel1 * sel2
dtmp = agn[sel]
sel = np.argsort(-dtmp['nmem'])
dtmp[sel]

agnid,ra,dec,z,z_er,zflag,field,nshots,detectid_best,ra_best,dec_best,roff,nmem,alpha,alpha_er,fpl0,fpl0_er,slope,slope_er,intercept,intercept_er,L1350,L1350_er,L1450,L1450_er,L3000,L3000_er,L5100,L5100_er,fwhm_LyA,fwhm_LyA_er,ew_LyA,ew_LyA_er,flux_LyA,flux_LyA_er,cont_LyA,cont_LyA_er,snr_LyA,snr_LyA_er,chi2_LyA,chi2_LyA_er,fwhm_NV,fwhm_NV_er,ew_NV,ew_NV_er,flux_NV,flux_NV_er,cont_NV,cont_NV_er,fwhm_SiO,fwhm_SiO_er,ew_SiO,ew_SiO_er,flux_SiO,flux_SiO_er,cont_SiO,cont_SiO_er,snr_SiO,snr_SiO_er,chi2_SiO,chi2_SiO_er,fwhm_CIV,fwhm_CIV_er,ew_CIV,ew_CIV_er,flux_CIV,flux_CIV_er,cont_CIV,cont_CIV_er,snr_CIV,snr_CIV_er,chi2_CIV,chi2_CIV_er,fwhm_HeII,fwhm_HeII_er,ew_HeII,ew_HeII_er,flux_HeII,flux_HeII_er,cont_HeII,cont_HeII_er,snr_HeII,snr_HeII_er,chi2_HeII,chi2_HeII_er,fwhm_CIII,fwhm_CIII_er,ew_CIII,ew_CIII_er,flux_CIII,flux_CIII_er,cont_CIII,cont_CIII_er,snr_CIII,snr_CIII_er,chi2_CIII,chi2_CIII_er,fwhm_MgII,fwhm_MgII_er,ew_MgII,ew_MgII_er,flux_MgII,flux_MgII_er,cont_MgII,cont_MgII_er,snr_MgII,snr_MgII_er,chi2_MgII,chi2_MgII_er,fwhm_OII,fwhm_OII_er,ew_OII,ew_OII_er,flux_OII,flux_OII_er,cont_OII,cont_OII_er,snr_OII,snr_OII_er,chi2_OII,chi2_OII_er,fwhm_OVI,fwhm_OVI_er,ew_OVI,ew_OVI_er,flux_OVI,flux_OVI_er,cont_OVI,cont_OVI_er,snr_OVI,snr_OVI_er,chi2_OVI,chi2_OVI_er,fiberid,r_aper,r_aper_er,r_radius,r_depth,r_cat,g_aper,g_aper_er,g_radius,g_depth,g_cat,u_aper,u_aper_er,u_radius,u_depth,u_cat,rsep_dr14q,id_dr14q,z_dr14q,rsep_dr16,plate_dr16,fiber_dr16,mjd_dr16,z_dr16,sflag,apcor
,deg,deg,,,,,,,deg,deg,arcsec,,,,,,,,,,1e+44 erg / s,1e+44 erg / s,1e+44 erg / s,1e+44 erg / s,1e+44 erg / s,1e+44 erg / s,1e+44 erg / s,1e+44 erg / s,km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),,,,,km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),,,,,km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),,,,,km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),,,,,km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),,,,,km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),,,,,km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),,,,,km / s,km / s,Angstrom,Angstrom,1e-17 erg / (s cm2),1e-17 erg / (s cm2),1e-17 erg / (Angstrom s cm2),1e-17 erg / (Angstrom s cm2),,,,,,mag,mag,arcsec,arcsec,,mag,mag,arcsec,arcsec,,mag,mag,arcsec,arcsec,,arcsec,,,arcsec,,,,,,
int64,float32,float32,float32,float32,int64,bytes10,int64,int64,float32,float32,float32,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes38,flo

### take agnid = 3194 as an example

In [13]:
agnid = 3194

im_arr    = hdul[5].data
im_er_arr = hdul[6].data

sel = np.where( tab['agnid'] == agnid )
shotid_arr  = tab[sel]['shotid']
flux_arr    = im_arr[sel]
flux_er_arr = im_er_arr[sel]

# arrange the repeat observations by time
sel = np.argsort( shotid_arr )
shotid_arr, flux_arr, flux_er_arr = shotid_arr[sel], flux_arr[sel], flux_er_arr[sel]

# prepare the wavelength array for the spectra
wave_start = 3470.0
wave_step  = 2.0
n_elements = hdul[5].header['NAXIS1'] #1036
wave_arr = wave_start + wave_step * np.arange( 0, n_elements )

# plot all repeat observations
fig, ax = plt.subplots( figsize = ( 10., 5.)  )

for i in range(np.size(shotid_arr)):
    
    plt.plot( wave_arr, flux_arr[i], label = '%d' % shotid_arr[i])
    #plt.errorbar( wave_arr, flux_arr[i], flux_er_arr[i] , capsize = 1.5, label = '%d' % shotid_arr[i] )

# overplot with the catalog spectrum for detectid_best in ext 2&3 as black and compare with ext 5&6
im_arr    = hdul[2].data
im_er_arr = hdul[3].data
sel = agn['agnid'] == agnid
flux_arr    = im_arr[sel][0]
flux_er_arr = im_er_arr[sel][0]
plt.errorbar( wave_arr, flux_arr, flux_er_arr, color = 'k',capsize = 1., \
              label = 'ext2&3 spectrum')
    
plt.xlim( xl, xh )
plt.tick_params( which = 'both', bottom=True, top=True, left=True, right=True )

plt.xlabel('$\mathrm{\lambda_{obs}\ (\AA)}$',fontsize=fontsize)
plt.ylabel('$\mathrm{f_{\lambda,obs}\ (10^{-17} ergs/s/cm^2/\AA)}$',fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.legend(fontsize=fontsize)
stitle = 'agnid=%d' % (agnid)
plt.title( stitle, fontsize=fontsize )
plt.show()